# Notebook for data cleaning tests

In [6]:
import requests
from io import BytesIO
import pandas as pd
import re
import unicodedata

pd.set_option('display.max_columns', None) # Option to force pandas to print every columns (useful for data exploration)

# Web archive URL to keep prevent URL change
DATA_URL = "https://web.archive.org/web/20240423194012/https://naehrwertdaten.ch/wp-content/uploads/2023/08/Base_de_donnees_suisse_des_valeurs_nutritives.xlsx"

CONVERSION_FACTORS = {
    "g": 1,
    "mg": 0.001,
    "kj": 1000,
    "µg": 0.000001,
    "kcal": 1000
}

def fetch_data(URL):
    response = requests.get(DATA_URL, stream=True)
    response.raise_for_status()
    data = BytesIO(response.content)
    return data

def remove_accents(text):
    normalized_text = unicodedata.normalize('NFKD', text)
    return normalized_text.encode('ascii', 'ignore').decode('utf-8')

def clean_data(df: pd.DataFrame):
    df = df.drop_duplicates()
    df = df.drop(columns=["ID V 4.0", "ID SwissFIR", "Densité", "Entrée modifiée"])
    df = df.drop(df.filter(regex=r'^Source.*').columns, axis=1)
    df = df.drop(df.filter(regex=r'^Dérivation de la valeur.*').columns, axis=1)
    df = df.drop(df.filter(regex=r'^Activité de *').columns, axis=1)

    double_parenthesis_pattern = r'\s*\([^)]+\)(?=\s*\([^)]+\))'
    
    df.columns = df.columns.str.replace(double_parenthesis_pattern, '', regex=True)
    df.columns = [remove_accents(col.strip().replace(',', '').replace(' ', '_').replace("-", "_").lower()) for col in df.columns]
    return df

def create_measures_table(df: pd.DataFrame):
    measures = []
    columns = df.columns
    for col in columns:
        match = re.search(r"\((.*?)\)", col)
        if match:
            measures.append({
                "name": re.sub(r"\((.*?)\)", "", col).strip("_"),
                "unit": match.group(1),
                "conversion": CONVERSION_FACTORS[match.group(1)]
            })

    return pd.DataFrame(measures)
        
    

raw_data = pd.read_excel("swiss_data.xlsx", engine = "openpyxl", skiprows=2)
data = clean_data(raw_data)
measures_table = create_measures_table(data)
data.columns = [re.sub(r"\((.*?)\)", "", col).strip("_") for col in data.columns]

,conversion
count,37.000000
mean,54.568000
std,229.119409
min,0.001000
25%,0.001000
50%,1.000000
75%,1.000000
max,1000.000000
